In [ ]:
# Multi-Agent Job Application Optimization System

## Problem Statement

**Problem**: Job applications are complex, multi-step processes requiring specialized expertise across different domains. Candidates must research job requirements, compile comprehensive profiles, tailor resumes to specific roles, and prepare for interviews - each requiring different skills and knowledge that most job seekers lack.

**Why AI Agents**: AI agents can process large amounts of information from multiple sources (job postings, GitHub profiles, personal data), apply specialized knowledge consistently, and provide personalized recommendations at scale.

**Multi-Agent Value**: Instead of one agent trying to handle everything, we use specialized agents that work together:
- Each agent focuses on their expertise area with specific tools
- Agents can process different data sources simultaneously  
- Collaborative workflow ensures each step builds on the previous one
- Context sharing between agents creates comprehensive, personalized results

**Solution**: A CrewAI-based system with 4 specialized agents that work together to optimize the entire job application process:
- **Tech Job Researcher**: Scrapes and analyzes job postings to extract key requirements, skills, and qualifications
- **Personal Profiler**: Compiles comprehensive candidate profiles from GitHub repositories and personal information
- **Resume Strategist**: Tailors resumes to highlight relevant skills and experiences that match job requirements
- **Interview Preparer**: Creates targeted interview questions and talking points based on the tailored resume and job requirements

**Tools Used**: SerperDev (web search), ScrapeWebsite (job posting analysis), FileRead (resume processing), MDXSearch (semantic search of candidate data)


In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew

In [ ]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [ ]:
from crewai_tools import (
  FileReadTool,
  ScrapeWebsiteTool,
  MDXSearchTool,
  SerperDevTool
)

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()
read_resume = FileReadTool(file_path='./fake_resume.md')
semantic_search_resume = MDXSearchTool(mdx='./fake_resume.md')

## agent creation

In [ ]:
# Agent 1: Researcher
researcher = Agent(
    role="Job Market Intelligence Analyst",
    goal="Conduct deep analysis of job postings to identify key requirements, skills, and qualifications that candidates need to highlight",
    tools = [scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "You are a seasoned tech industry analyst with 8+ years of experience in talent acquisition. "
        "Your expertise lies in decoding job descriptions, identifying hidden requirements, and understanding "
        "what hiring managers truly value. You excel at extracting not just the obvious qualifications, "
        "but also the underlying skills and experiences that separate top candidates from average ones."
    )
)

In [ ]:
# Agent 2: Profiler
profiler = Agent(
    role="Candidate Intelligence Specialist",
    goal="Build comprehensive professional profiles by analyzing GitHub activity, project portfolios, and career history to identify unique strengths and achievements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "You are a former technical recruiter turned career strategist with deep expertise in evaluating "
        "software engineers. Your background includes 6 years at top tech companies where you've reviewed "
        "thousands of profiles. You have a unique ability to identify transferable skills, quantify achievements, "
        "and spot patterns in career progression that others miss. You understand both technical depth and "
        "business impact, making you exceptional at translating complex technical work into compelling narratives."
    )
)

In [ ]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Technical Resume Architect",
    goal="Transform candidate profiles into compelling, ATS-optimized resumes that strategically align with specific job requirements",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "You are a former engineering manager and technical writing expert with 10+ years of experience in "
        "helping engineers advance their careers. Your unique approach combines technical accuracy with "
        "marketing psychology - you know how to make complex technical achievements sound impressive to "
        "both technical and non-technical audiences. You've helped over 500 engineers land roles at "
        "companies like Google, Microsoft, and startups, and you understand the subtle differences "
        "between what different types of companies value in resumes."
    )
)

In [ ]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Technical Interview Coach",
    goal="Develop targeted interview strategies, technical questions, and behavioral talking points that showcase the candidate's strengths and address potential concerns",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_resume],
    verbose=True,
    backstory=(
        "You are a former engineering manager and technical interviewer with 12+ years of experience conducting "
        "interviews at companies ranging from startups to FAANG. You've interviewed over 1,000 candidates and "
        "understand the psychology behind technical interviews. Your expertise includes behavioral questions, "
        "system design, coding challenges, and cultural fit assessments. You know how to help candidates "
        "anticipate what interviewers are really looking for and how to present their experience in the most "
        "compelling way possible."
    )
)

# task creation

In [ ]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Conduct a comprehensive analysis of the job posting at {job_posting_url}. "
        "Extract both explicit and implicit requirements, including technical skills, "
        "soft skills, experience levels, and cultural fit indicators. Identify the "
        "most critical qualifications that will differentiate successful candidates. "
        "Use web scraping and search tools to gather additional context about the "
        "company and role expectations."
    ),
    expected_output=(
        "A detailed breakdown of job requirements organized by priority, including "
        "must-have qualifications, nice-to-have skills, and hidden requirements "
        "that indicate what the company truly values. Include specific examples "
        "and context for each requirement."
    ),
    agent=researcher,
    async_execution=True
)

In [ ]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Build a comprehensive professional profile by analyzing the candidate's "
        "GitHub repository at {github_url} and personal background from {personal_writeup}. "
        "Examine code quality, project complexity, collaboration patterns, and technical "
        "expertise demonstrated through their work. Identify unique strengths, "
        "achievements, and areas of specialization that set them apart from other candidates."
    ),
    expected_output=(
        "A detailed professional profile including technical skills assessment, "
        "project portfolio analysis, collaboration patterns, code quality evaluation, "
        "unique strengths identification, and quantified achievements. Include specific "
        "examples and metrics that demonstrate the candidate's capabilities."
    ),
    agent=profiler,
    async_execution=True
)

In [ ]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Transform the candidate's profile into a compelling, ATS-optimized resume "
        "that strategically aligns with the job requirements. Focus on highlighting "
        "the most relevant skills and experiences while maintaining authenticity. "
        "Optimize for both human recruiters and ATS systems, ensuring keyword density "
        "and proper formatting. Emphasize quantifiable achievements and impact."
    ),
    expected_output=(
        "A professionally tailored resume that effectively showcases the candidate's "
        "qualifications and experiences relevant to the specific job. The resume should "
        "be ATS-friendly, well-formatted, and highlight key achievements with metrics. "
        "Include a compelling professional summary and optimized skills section."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [ ]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Develop comprehensive interview preparation materials based on the tailored "
        "resume and job requirements. Create targeted technical questions, behavioral "
        "scenarios, and talking points that allow the candidate to showcase their "
        "strengths and address potential concerns. Include both general and role-specific "
        "questions that interviewers are likely to ask."
    ),
    expected_output=(
        "A comprehensive interview preparation guide including technical questions, "
        "behavioral scenarios, STAR method examples, and strategic talking points. "
        "Include questions about specific technologies, projects, and experiences "
        "mentioned in the resume, along with suggested responses that highlight "
        "the candidate's unique value proposition."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)


# Crew creation

In [ ]:
job_application_crew = Crew(
    agents=[researcher, profiler, resume_strategist, interview_preparer],
    tasks=[research_task, profile_task, resume_strategy_task, interview_preparation_task],
    process=Process.sequential, 
    verbose=True,
    memory=True, 
   
)


In [ ]:
# running the crew

In [ ]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.lever.co/AIFund/6c82e23e-d954-4dd8-a734-c0c2c5ee00f1?lever-origin=applied&lever-source%5B%5D=AI+Fund',
    'github_url': 'https://github.com/joaomdmoura',
    'personal_writeup': """Noah is an accomplished Software
    Engineering Leader with 18 years of experience, specializing in
    managing remote and in-office teams, and expert in multiple
    programming languages and frameworks. He holds an MBA and a strong
    background in AI and data science. Noah has successfully led
    major tech initiatives and startups, proving his ability to drive
    innovation and growth in the tech industry. Ideal for leadership
    roles that require a strategic and innovative approach."""
}

In [ ]:
# Execute the complete analysis workflow
result = run_job_application_analysis(job_application_inputs)

In [ ]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

In [ ]:
display(Markdown("./interview_materials.md"))